<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Soluci%C3%B3n.%20Generador%20de%20Res%C3%BAmenes%20Inteligente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1. Configuración del entorno del cuaderno**
---
En esta celda se realiza la configuración inicial del entorno:
- Se importa la librería `userdata` de Google Colab para acceder a claves API almacenadas de forma segura.
- Se obtienen las claves API de servicios como OpenAI, Groq, Google GenAI y Hugging Face.
- Se instalan las librerías necesarias, como `langchain` y sus integraciones con diferentes proveedores de modelos de lenguaje (LLMs).
- Se importan las clases de LangChain para crear plantillas de prompts (`ChatPromptTemplate`, `SystemMessagePromptTemplate`, `HumanMessagePromptTemplate`) y para interactuar con los LLMs (`ChatOpenAI`, `ChatGroq`, etc.).
- También se importa `Markdown` de `IPython.display` para mejorar la presentación de texto en el notebook.

In [ ]:
# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
!pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
!pip install langchain-openai -qU
!pip install langchain-groq -qU
!pip install langchain-google-genai -qU
!pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.

from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint


from IPython.display import Markdown

## **2. Configuración del modelo de OpenAI**
---
Aquí se configura el modelo de OpenAI (`gpt-4o`) utilizando la clave API obtenida previamente. Se establece un parámetro de `temperature` en 0.7 para controlar la creatividad de las respuestas del modelo.

In [ ]:
llm = ChatOpenAI(model="gpt-4o",
                 api_key=OPENAI_API_KEY,
                 temperature=0.7)

##**3. Definimos las plantilla para los mensajes**
---
Se definen dos plantillas de prompts:
1. **`template_sistema`:** Instrucciones detalladas para el modelo, explicando su tarea de generar dos tipos de resúmenes (breve y detallado) a partir de un texto dado.
2. **`template_usuario`:** Plantilla para el mensaje del usuario, que incluye el texto que se desea resumir.

In [ ]:
# Definir las plantillas
template_sistema = """\
Eres un asistente experto en resumir textos largos. Tu tarea es generar dos tipos de resúmenes para el texto que recibas:

1. Resumen breve: Un resumen conciso de 3 a 4 oraciones que capture las ideas principales del texto.
2. Resumen detallado: Un resumen más extenso, de al menos un párrafo, que incluya detalles clave, ejemplos o puntos importantes del texto.

Instrucciones:
- Lee siempre el texto cuidadosamente.
- Identifica las ideas principales y los detalles relevantes.
- Genera primero el resumen breve y luego el resumen detallado.
- Asegúrate de que ambos resúmenes sean claros, coherentes y fieles al contenido original.

Ejemplo de formato de salida:
Resumen breve:   El cambio climático es un desafío global que causa el aumento de temperaturas, el derretimiento de glaciares y la subida del nivel del mar. Sus efectos impactan negativamente en los ecosistemas y las comunidades humanas. Para mitigarlo, es crucial reducir las emisiones y adoptar energías renovables.
Resumen detallado: El cambio climático representa uno de los problemas más urgentes de la actualidad, caracterizado por el aumento de las temperaturas globales, el derretimiento acelerado de los glaciares y el incremento del nivel del mar. Estos fenómenos tienen consecuencias graves, como la pérdida de biodiversidad, la destrucción de ecosistemas y el desplazamiento de comunidades humanas. Para enfrentar este desafío, es fundamental implementar medidas como la reducción de emisiones de gases de efecto invernadero, la transición hacia energías renovables y la adopción de prácticas sostenibles en la industria, la agricultura y otros sectores clave.


"""

template_usuario = """\
Texto a resumir: {mensaje_usuario}

"""

## **4. Creación del ChatPromptTemplate**
---
Se crea un `ChatPromptTemplate` utilizando las plantillas definidas en la celda anterior. Este template combina las instrucciones del sistema (`template_sistema`) y el mensaje del usuario (`template_usuario`) en un formato que el modelo puede entender.

In [ ]:
# Crear el ChatPromptTemplate
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", template_sistema),
    ("human", template_usuario),
])

## **5. Pedimos el texto a resumir**
---

In [ ]:
texto_input = input("Escribe aquí el texto a resumir")

##**6. Generación de resúmenes**
---
Formateamos el ChatPromptTemplate e invocamos al modelo pasandoselo
Presentamos la respuestas.

In [ ]:
    # Formatear el prompt con el mensaje del usuario y el idioma
    prompt = chat_prompt_template.format(
        mensaje_usuario=texto_input,

    )

    # Obtener la respuesta del modelo
    respuesta = llm.invoke(prompt)
    display(Markdown(respuesta.content))